In [1]:
for name in dir():
    if not name.startswith('_'):
        del globals()[name]

In [2]:
import os
import random
import PIL
import numpy as np # linear algebra
import matplotlib.pyplot as plt # Ploting the diagrams

from sklearn.utils import shuffle

from PIL import ImageOps
from IPython.display import Image, display
import math
from sklearn.model_selection import StratifiedKFold #look into stratified kfolds
# Importing tensorflow and keras
import tensorflow as tf
from tensorflow.keras import regularizers
from tensorflow import keras
from keras import layers
from keras.layers import Softmax, Conv2D, MaxPooling2D, BatchNormalization, Activation, UpSampling2D, SeparableConv2D, Conv2DTranspose
from keras.layers import add
from keras.preprocessing.image import load_img

import os
from os import listdir
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import matplotlib.pyplot as plt # Ploting the diagrams
import matplotlib.image as mpimg

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
import tensorflow as tf
from tensorflow import keras
from keras.models import model_from_json

from tensorflow.keras.layers import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from tensorflow.keras import datasets, layers, models
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten,BatchNormalization,Activation,Rescaling
import random
from sklearn.pipeline import Pipeline
from keras.models import model_from_json
import pandas as pd
tf.random.set_seed(333)



In [3]:
def genImageList(l):
    return list(filter(lambda i: i.split(".")[-1] in ["png","jpg","jpeg"], l))
def genDf(images,labels):
    df= pd.DataFrame(images,columns=["Image"])
    df["Class"] = labels
    return df

In [4]:
def evaluate(df,models):
    gMaskPath = r'Test/Mask/'
    gNoMaskPath = r'Test/NoMask/'
    gMaskImages = genImageList([gMaskPath+i for i in os.listdir(gMaskPath)])
    gMaskLabels = np.zeros(len(gMaskImages))
    gNoMaskImages = genImageList([gNoMaskPath+i for i in os.listdir(gNoMaskPath)])
    gNoMaskLabels = np.ones(len(gNoMaskImages))

    gMaskDf = genDf(gMaskImages,gMaskLabels)
    gNoMaskDf = genDf(gNoMaskImages,gNoMaskLabels)

    df = pd.DataFrame(shuffle(pd.concat([gMaskDf,gNoMaskDf])),columns=gMaskDf.columns)
    
    x,y = df["Image"],df["Class"]
    data = []
    size = 100
    
    for img_path in x:
        image = load_img(img_path, target_size=(size, size))
        image = img_to_array(image)
        image = image/255
        data.append(image)

    data = np.array(data, dtype="float32")
    labels = np.array(y)
    return data,labels
tf.keras.backend.clear_session()

def evaluationData():
    gMaskPath = r'Test/Mask/'
    gNoMaskPath = r'Test/NoMask/'
    gMaskImages = genImageList([gMaskPath+i for i in os.listdir(gMaskPath)])
    gMaskLabels = np.zeros(len(gMaskImages))
    gNoMaskImages = genImageList([gNoMaskPath+i for i in os.listdir(gNoMaskPath)])
    gNoMaskLabels = np.ones(len(gNoMaskImages))

    gMaskDf = genDf(gMaskImages,gMaskLabels)
    gNoMaskDf = genDf(gNoMaskImages,gNoMaskLabels)

    df = pd.DataFrame(shuffle(pd.concat([gMaskDf,gNoMaskDf])),columns=gMaskDf.columns)
    
    x,y = df["Image"],df["Class"]
    data = []
    size = 100
    
    for img_path in x:
        image = load_img(img_path, target_size=(size, size))
        image = img_to_array(image)
        image = image/255
        data.append(image)

    data = np.array(data, dtype="float32")
    labels = np.array(y)
    
    return data,labels

In [5]:
with open('LastRound.txt', 'r') as f:
    rounds = int(f.read())

rounds = 12
central = keras.models.load_model("C"+str(rounds)+".h5")
globalModel =  keras.models.load_model("G"+str(rounds)+".h5")
c =  [keras.models.load_model("C"+str(rounds)+str(i+1)+".h5") for i in range(5)]

models = c+[globalModel,central]



data,labels = evaluationData()

testDic = {}

modelNames =  ["Client"+str(i+1) for i in range(5)] +["Global Model","Central Model"]

for i in range(7):
    name = modelNames.pop(0)
    print(name)
    testDic[name] = models[0].evaluate(data,labels)[-1]
    models.pop(0)
    tf.keras.backend.clear_session()

Client1
188/188 [==============================] - 3s 6ms/step - loss: 0.2536 - recall_1: 0.9532 - precision_1: 0.9385 - auc_1: 0.9725 - accuracy: 0.9454
Client2
188/188 [==============================] - 1s 6ms/step - loss: 0.2200 - recall_2: 0.8891 - precision_2: 0.9521 - auc_2: 0.9709 - accuracy: 0.9222
Client3
188/188 [==============================] - 1s 5ms/step - loss: 0.3366 - recall_3: 0.9836 - precision_3: 0.9034 - auc_3: 0.9674 - accuracy: 0.9392
Client4
188/188 [==============================] - 1s 6ms/step - loss: 0.3204 - recall_4: 0.9866 - precision_4: 0.8847 - auc_4: 0.9707 - accuracy: 0.9290
Client5
188/188 [==============================] - 1s 5ms/step - loss: 0.2744 - recall_5: 0.9559 - precision_5: 0.9452 - auc_5: 0.9675 - accuracy: 0.9502
Global Model
188/188 [==============================] - 1s 6ms/step - loss: 0.3286 - recall: 0.9689 - precision: 0.9268 - auc: 0.9686 - accuracy: 0.9462
Central Model
188/188 [==============================] - 1s 6ms/step - loss: 

In [6]:
import pickle
with open('Testing_Rounds_'+str(rounds)+'.pkl', 'wb') as f:
    pickle.dump(testDic, f)